## Data processed

In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import MACCSkeys
import numpy as np
import torch
import torch.nn as nn
import torch.utils
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
from tqdm import tqdm
import os
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from SLIMNet import SLIMNet, train_SLIMNet, GetLoader
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.offsetbox import AnchoredText
from gensim.models import word2vec
from mol2vec.features import mol2alt_sentence, MolSentence, DfVec, sentences2vec
from mol2vec.helpers import depict_identifier, plot_2D_vectors, IdentifierTable, mol_to_svg
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole

def plot_parity(y_true, y_pred, name, dpi_num, y_pred_unc=None):
    axmin = min(min(y_true), min(y_pred)) - 0.1*(max(y_true)-min(y_true))
    axmax = max(max(y_true), max(y_pred)) + 0.1*(max(y_true)-min(y_true))
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    sns.set()
    plt.plot([axmin, axmax], [axmin, axmax], '--k')

    plt.errorbar(y_true, y_pred, yerr=y_pred_unc, linewidth=0, marker='o', markeredgecolor='w', c = 'cornflowerblue', alpha=1, elinewidth=1)
    plt.xlim((axmin, axmax))
    plt.ylim((axmin, axmax))
    ax = plt.gca()
    # ax.set_aspect('equal')

    # at = AnchoredText(
    #     f"MAE = {mae:.3f}\nRMSE = {rmse:.3f}\nR^2 = {r2:.3f}", prop=dict(size=10), frameon=True, loc='upper left')
    # at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
    # ax.add_artist(at)
    ax.spines['bottom'].set_linewidth(2);###设置底部坐标轴的粗细
    ax.spines['left'].set_linewidth(2);####设置左边坐标轴的粗细
    ax.spines['right'].set_linewidth(2);###设置右边坐标轴的粗细
    ax.spines['top'].set_linewidth(2);####设置上部坐标轴的粗细
    # plt.xlabel('Ground-Truth')
    # plt.ylabel('Predicted')
    plt.savefig(f'{name}.png', dpi = dpi_num)

    plt.show()
    return


def plot_comp(y_t, y_p, name, dpi_num, color, y_pred_unc=None):
    
    plt.figure(figsize=(3, 3),dpi=500)
    #sns.set(style='white')
    # color = ['#2878b5', '#9ac9db', '#f8ac8c', '#c82423']
    # color = ['#8ECFC9' ,'#FFBE7A', '#FA7F6F', '#82B0D2']
    # color = ['#A1A9D0','#F0988C','#B883D4','#9E9E9E']
    # color = ['#F27970','#BB9727','#54B345' ,'#32B897']
    # color = ['#db3124', '#ffdf92', '#90bee0', '#4b74b2']
    color = color #'#db3124'
    
    # plt.subplot(2, 2 ,i+1)
    y_true = y_t
    y_pred = y_p
    axmin = min(min(y_true), min(y_pred)) - 0.1*(max(y_true)-min(y_true))
    axmax = max(max(y_true), max(y_pred)) + 0.1*(max(y_true)-min(y_true))
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    plt.plot([axmin, axmax], [axmin, axmax], '--k')
    plt.errorbar(y_true, y_pred, yerr=y_pred_unc, linewidth=0, marker='o', markeredgecolor='w', c = color, alpha=1, elinewidth=1)
    plt.xlim((axmin, axmax))
    plt.ylim((axmin, axmax))
    plt.yticks(size = 17)
    plt.xticks(size = 17)
    ax = plt.gca()
    ax.spines['bottom'].set_linewidth(2);###设置底部坐标轴的粗细
    ax.spines['left'].set_linewidth(2);####设置左边坐标轴的粗细
    ax.spines['right'].set_linewidth(2);###设置右边坐标轴的粗细
    ax.spines['top'].set_linewidth(2);####设置上部坐标轴的粗细
    #plt.xlabel(f'Ground-Truth of {i}')
    #plt.ylabel(f'Predicted {i}')
    # sns.despine()

    plt.savefig(f'{name}.png', dpi = dpi_num)
    plt.show()

## Data process

In [2]:
data = pd.read_csv('PI1070.csv')
data = data.fillna(0)
maccs = []

model = word2vec.Word2Vec.load('model.pkl')
data['sentence'] = data.apply(lambda x: MolSentence(mol2alt_sentence(Chem.MolFromSmiles(x['smiles']), 1)), axis=1)
data['mol2vec'] = [DfVec(x) for x in sentences2vec(data['sentence'], model, unseen='UNK')]
X0 = np.array([x.vec for x in data['mol2vec']])
#X0 = np.array(maccs).astype('int') # (1077, 300)
X1 = np.array(data['mol_weight_monomer']).reshape(-1,1) #(1077, 1)
X2 = np.array(data['vdw_volume_monomer']).reshape(-1,1) #(1077, 1)
X3 = np.array(data['qm_total_energy_monomer']).reshape(-1,1) #(1077, 1)
X4 = np.array(data['qm_homo_monomer']).reshape(-1,1) #(1077, 1)
X5 = np.array(data['qm_lumo_monomer']).reshape(-1,1) #(1077, 1)
X6 = np.array(data['qm_dipole_monomer']).reshape(-1,1) #(1077, 1)
X7 = np.array(data['qm_polarizability_monomer']).reshape(-1,1) #(1077, 1)
X8 = np.array(data['DP']).reshape(-1,1) #(1077, 1)  
X9 = np.array(data['Mn']).reshape(-1,1) #(1077, 1)
X10 = np.array(data['density']).reshape(-1,1) #(1077, 1)
X11 = np.array(data['compressibility']).reshape(-1,1) #(1077, 1)
X12 = np.array(data['static_dielectric_const']).reshape(-1,1) #(1077, 1)
X13 = np.array(data['nematic_order_parameter']).reshape(-1,1) #(1077, 1)

y0 = np.array(data['thermal_conductivity']).reshape(-1,1)
y1 = np.array(data['thermal_diffusivity']).reshape(-1,1) #(1077, 1)
y2 = np.array(data['dielectric_const_dc']).reshape(-1,1) #(1077, 1)
y3 = np.array(data['volume_expansion']).reshape(-1,1) #(1077, 1)

y = np.concatenate((y0, y1, y2, y3), axis=1)
X = np.concatenate((X0, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13), axis=1)



[22:29:18] DEPRECATION WARNING: please use MorganGenerator
[22:29:18] DEPRECATION WARNING: please use MorganGenerator
[22:29:18] DEPRECATION WARNING: please use MorganGenerator
[22:29:18] DEPRECATION WARNING: please use MorganGenerator
[22:29:18] DEPRECATION WARNING: please use MorganGenerator
[22:29:18] DEPRECATION WARNING: please use MorganGenerator
[22:29:18] DEPRECATION WARNING: please use MorganGenerator
[22:29:18] DEPRECATION WARNING: please use MorganGenerator
[22:29:18] DEPRECATION WARNING: please use MorganGenerator
[22:29:18] DEPRECATION WARNING: please use MorganGenerator
[22:29:18] DEPRECATION WARNING: please use MorganGenerator
[22:29:18] DEPRECATION WARNING: please use MorganGenerator
[22:29:18] DEPRECATION WARNING: please use MorganGenerator
[22:29:18] DEPRECATION WARNING: please use MorganGenerator
[22:29:18] DEPRECATION WARNING: please use MorganGenerator
[22:29:18] DEPRECATION WARNING: please use MorganGenerator
[22:29:18] DEPRECATION WARNING: please use MorganGenerat

In [3]:
# import numpy as np
# 
# train_id = np.random.choice(1077, 862, replace=False)
# # train_id = np.load('train_id.npy')
# test_id = np.setdiff1d(np.arange(1077), train_id, assume_unique=False)
# # print(train_id)
# print(train_id.shape)
# # print(train_id)
# print(test_id.shape)
# np.save('train_id.npy', train_id)
# np.save('test_id.npy', test_id)

## training of slimnet

In [3]:
model = SLIMNet(4)
scalar = StandardScaler()
X = scalar.fit_transform(X)
y = scalar.fit_transform(y)

train_id = np.load('../slim_maccs/best_pt/train_id.npy')
test_id = np.load('../slim_maccs/best_pt/test_id.npy')

X_train, X_test = X[train_id], X[test_id]
y_train, y_test = y[train_id], y[test_id]
train = 0

if train: 
    train_data, test_data = GetLoader(X_train, y_train), GetLoader(X_test, y_test)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=False)
    train_SLIMNet(model, 7000, train_loader, test_loader, X_test, y_test,4, 10)

In [ ]:
model = SLIMNet(4)
path = f'softnet-test_k.pt'
net_para_evidential = torch.load(path, map_location='cpu')
model.load_state_dict(net_para_evidential['model_state_dict'])
# model.load_state_dict(torch.load("swa_model.pt"))
model.eval()
print(torch.tensor(X_test).shape)

monomer_prop, scal, alpha, chain_order = model(torch.tensor(X_test).float())
beta = scal[:, :4]
gamma = scal[:, 4:]
mid = torch.pow(gamma, beta)
prop_guass = torch.mul(alpha, mid)
polymer_prop = chain_order + prop_guass
y_predict_softnet = polymer_prop.detach().numpy()

print(f'MSE: {mean_squared_error(y_test, y_predict_softnet)}')
print(f'MAE: {mean_absolute_error(y_test, y_predict_softnet)}')
print(f'R2: {r2_score(y_test, y_predict_softnet)}')
print(y_test.shape)

plot_comp(np.mean(y_test, axis=-1),np.mean(y_predict_softnet,axis=-1),'softnet_test', 1000, color='cornflowerblue') ##db3124, '#ffdf92', '#90bee0', '#4b74b2'


## training of regular_NN

In [5]:
from NN import regular_NN, train_regular_NN, GetLoader
model = regular_NN(4)
scalar = StandardScaler()
X = scalar.fit_transform(X)
y = scalar.fit_transform(y)

train_id = np.load('../slim_maccs/best_pt/train_id.npy')
test_id = np.load('../slim_maccs/best_pt/test_id.npy')

X_train, X_test = X[train_id], X[test_id]
y_train, y_test = y[train_id], y[test_id]
train = 0

if train: 
    train_data, test_data = GetLoader(X_train, y_train), GetLoader(X_test, y_test)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=False)
    train_regular_NN(model, 7000, train_loader, test_loader, X_test, y_test,4, 10)

In [ ]:
model = regular_NN(4)

path = f'regularnn-test.pt'
net_para_evidential = torch.load(path, map_location='cpu')
model.load_state_dict(net_para_evidential['model_state_dict'])
# model.load_state_dict(torch.load("swa_model.pt"))
model.eval()
print(torch.tensor(X_test).shape)

monomer_prop = model(torch.tensor(X_test).float())

polymer_prop = monomer_prop
y_predict_softnet = polymer_prop.detach().numpy()

print(f'MSE: {mean_squared_error(y_test, y_predict_softnet)}')
print(f'MAE: {mean_absolute_error(y_test, y_predict_softnet)}')
print(f'R2: {r2_score(y_test, y_predict_softnet)}')
print(y_test.shape)

plot_comp(np.mean(y_test, axis=-1),np.mean(y_predict_softnet,axis=-1),'regularnn_test', 1000, color='#f1d77e')#f3d266